In [3]:
def write_label_map(label_map, output_path):
    with open(output_path, 'w') as f:
        for id, name in label_map.items():
            f.write('item {\n')
            f.write('  id: {}\n'.format(id))
            f.write("  name: '{}'\n".format(name))
            f.write('}\n')

In [4]:
label_map = {
    1: 'Car',
    2: 'Truck'
}

write_label_map(label_map, '.\\training_data\\labels\\coco_labels.pbtxt')

In [5]:
#cropping images to test
from PIL import Image

def crop_image(image_name: str, xmin: int, ymin: int, xmax: int, ymax: int):
    # Load the image
    image_path = ".\\training_data\\object-detection-crowdai\\" + image_name
    image = Image.open(image_path)
    
    # Define the bounding box
    bbox = (xmin, ymin, xmax, ymax)
    
    # Crop the image using the bounding box
    cropped_image = image.crop(bbox)
    
    # Save the cropped image
    cropped_image.save(".\\invalid_cropped_images\\" + image_name)

In [8]:
#CSV TO COCO JSON
import csv
import json
from PIL import Image

csv_file = '.\\training_data\\labels\\labels.csv'
json_file = '.\\training_data\\labels\\coco.json'

ignore_labels = ["Pedestrian"]
# Define the categories
categories = [
    {'id': 1, 'name': 'Car'},
    {'id': 2, 'name': 'Truck'}
]

# Initialize the data dictionary
data = {'images': [], 'annotations': [], 'categories': categories}

# Read the CSV file and add annotations to the data dictionary
with open(csv_file, 'r') as f:
    reader = csv.DictReader(f)
    counter = 1  # Initialize the counter variable
    for row in reader:
        if row["Label"] in ignore_labels:
            continue
        
        image_id = int(row['Frame'].split('.')[0])
        category_id = 0
        for category in categories:
            if row['Label'] == category['name']:
                category_id = category['id']
                break

        annotation = {
            'id': counter,
            'image_id': image_id,
            'category_id': category_id,
            'bbox': [int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])],
            'area': ((int(row['xmax']) - int(row['xmin'])) * (int(row['ymax']) - int(row['ymin']))),
            "iscrowd": 0
        }
        
        # if annotation['area'] < 0:
        #     crop_image(row['Frame'], int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']))

        data['annotations'].append(annotation)
        counter +=1

# Add the images to the data dictionary
for annotation in data['annotations']:
    if annotation['image_id'] not in [image['id'] for image in data['images']]:
        
        # image = Image.open('.\\training_data\\object-detection-crowdai\\' + str(annotation['image_id']) + '.jpg')
        # width, height = image.size
        
        data['images'].append({
            'id': annotation['image_id'],
            'file_name': str(annotation['image_id']) + '.jpg',
            'height': 1200, # Replace with the actual height of your images
            'width': 1920 # Replace with the actual width of your images
        })

# Save the data dictionary as JSON
with open(json_file, 'w') as f:
    json.dump(data, f)

In [ ]:
#run protoc to setup tenserflow repo
!protoc .\object_detection\protos\*.proto --python_out=.

In [10]:
import os
import subprocess
import pyperclip

train_image_dir = '..\\..\\..\\..\\..\\training_data\\object-detection-crowdai'
val_image_dir = '..\\..\\..\\..\\..\\training_data\\object-detection-crowdai'
test_image_dir = '..\\..\\..\\..\\..\\training_data\\object-detection-crowdai'
train_annotations_file = '..\\..\\..\\..\\..\\training_data\\labels\\coco.json'
val_annotations_file = '..\\..\\..\\..\\..\\training_data\\labels\\coco.json'
testdev_annotations_file = '..\\..\\..\\..\\..\\training_data\\labels\\coco.json'
output_dir = '..\\..\\..\\..\\..\\models\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\setup_files'
label_map_path = '..\\..\\..\\..\\..\\training_data\\labels\\coco_labels.pbtxt'

cmd = [
    'python', 'create_coco_tf_record.py', '--logtostderr', 
    f'--train_image_dir={train_image_dir}', 
    f'--val_image_dir={val_image_dir}',
    f'--test_image_dir={test_image_dir}',
    f'--train_annotations_file={train_annotations_file}',
    f'--val_annotations_file={val_annotations_file}',
    f'--testdev_annotations_file={testdev_annotations_file}',
    f'--output_dir={output_dir}',
    f'--label_map_path={label_map_path}'
]

cmd = f"python create_coco_tf_record.py --logtostderr --train_image_dir=\"{train_image_dir}\" --val_image_dir=\"{val_image_dir}\" --test_image_dir=\"{test_image_dir}\" --train_annotations_file=\"{train_annotations_file}\" --val_annotations_file=\"{val_annotations_file}\" --testdev_annotations_file=\"{testdev_annotations_file}\" --output_dir=\"{output_dir}\" --label_map_path=\"{label_map_path}\""

pyperclip.copy(cmd)
print('Command copied to clipboard.')

# subprocess.call(".\\tenserflow_env\\Scripts\\activate.bat && " + cmd, shell=True)
# result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# if result.returncode == 0:
#     print(result.stdout)
# else:
#     print(result.stderr)


Command copied to clipboard.


In [29]:
#merge coco record files

import tensorflow as tf
import os

def extract_data(record):
    feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/object/difficult': tf.io.VarLenFeature(tf.int64),
        'image/object/truncated': tf.io.VarLenFeature(tf.int64),
        'image/object/view': tf.io.VarLenFeature(tf.string),
        'image/object/is_crowd': tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(record, feature_description)
    return example

# Define a function to write each record to the output file
def write_to_tfrecord(example):
    with tf.io.TFRecordWriter(output_path) as writer:
        writer.write(tf.io.serialize_tensor(example))

current_dir = os.getcwd()
print(current_dir)

# os.chdir(".\\models\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\setup_files\\")

nr  = 50 # to be changed base on the number of tfrecords you have

# List all the sharded files
shard_filenames = ['coco_testdev.record-0000{}-of-000{}'.format(i, nr) for i in range(nr)]

# Load the dataset from the sharded files
dataset = tf.data.TFRecordDataset(shard_filenames).map(extract_data)
print(dataset)


# Define the output path for the combined .tfrecord file
output_path = 'coco_testdev.tfrecord'


# Map the dataset to the write function to create the output file
#dataset.map(write_to_tfrecord)


c:\Users\Mihai Gherghinescu\source\repos\TrafficManager\MLDataTraining\Python\models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\setup_files
<_MapDataset element_spec={'image/object/bbox/xmax': SparseTensorSpec(TensorShape([None]), tf.float32), 'image/object/bbox/xmin': SparseTensorSpec(TensorShape([None]), tf.float32), 'image/object/bbox/ymax': SparseTensorSpec(TensorShape([None]), tf.float32), 'image/object/bbox/ymin': SparseTensorSpec(TensorShape([None]), tf.float32), 'image/object/class/label': SparseTensorSpec(TensorShape([None]), tf.int64), 'image/object/class/text': SparseTensorSpec(TensorShape([None]), tf.string), 'image/object/difficult': SparseTensorSpec(TensorShape([None]), tf.int64), 'image/object/is_crowd': SparseTensorSpec(TensorShape([None]), tf.int64), 'image/object/truncated': SparseTensorSpec(TensorShape([None]), tf.int64), 'image/object/view': SparseTensorSpec(TensorShape([None]), tf.string), 'image/encoded': TensorSpec(shape=(), dtype=tf.string, name=None), 'image

To check if CUDA and cuDNN installed the right way

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
